In [1]:
!pip install rasterio


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 68.5 MB/s eta 0:00:00


In [2]:
import numpy as np
import rasterio
from scipy import stats
from skimage.feature import local_binary_pattern
import pywt

In [3]:
def extract_features(image_path):
    with rasterio.open(image_path) as src:
        image = src.read().astype(np.float32) / 10000 
    
    features = {}
    def calculate_ratio(band1, band2):
        return np.divide(band1, band2, where=band2!=0)

    # Ratio and statistical features
    features['ratio_band_1_8_mean_max_ratio'] = np.mean(calculate_ratio(image[0], image[7])) / np.max(calculate_ratio(image[0], image[7]))
    features['min_max_ratio_3'] = np.min(image[2]) / np.max(image[2])
    features['ratio_band_1_3_2_std'] = (np.std(calculate_ratio(image[0], image[2])) + np.std(calculate_ratio(image[0], image[1]))) / 2
    features['ratio_band_3_5_1_3_diff'] = np.mean(calculate_ratio(image[2], image[4])) - np.std(calculate_ratio(image[0], image[2]))
    features['ratio_band_2_4_1_2_diff'] = np.max(calculate_ratio(image[1], image[3])) - np.min(calculate_ratio(image[0], image[1]))
    features['ratio_band_2_3_mean_std_ratio'] = np.mean(calculate_ratio(image[1], image[2])) / np.std(calculate_ratio(image[1], image[2]))
    features['ratio_band_4_10_min_log'] = np.log1p(np.min(calculate_ratio(image[3], image[9])))
    features['ratio_band_1_8_max_mean_diff'] = np.max(calculate_ratio(image[0], image[7])) - np.mean(calculate_ratio(image[0], image[7]))
    features['ratio_band_3_5_1_3_std'] = np.std([calculate_ratio(image[2], image[4]), calculate_ratio(image[0], image[2])])
    features['band_1_2_3_4_correlation'] = np.corrcoef([image[0].flatten(), image[1].flatten(), image[2].flatten(), image[3].flatten()])[0,1]
    features['ratio_band_12_13_std'] = np.std(calculate_ratio(image[11], image[12]))
    features['band_6_13_std_ratio'] = np.std(image[5]) / np.std(image[12])
    features['ratio_band_1_7_13_max_std_ratio'] = np.max(calculate_ratio(image[0], image[6])) / np.std(calculate_ratio(image[0], image[12]))
    features['band_3_10_min_mean_diff'] = np.mean(image[2]) - np.min(image[9])
    features['ratio_band_1_9_std_min_diff'] = np.std(calculate_ratio(image[0], image[8])) - np.min(calculate_ratio(image[0], image[8]))
    features['ratio_band_3_5_mean'] = np.mean(calculate_ratio(image[2], image[4]))
    features['ratio_band_2_5_8_11_mean'] = np.mean([calculate_ratio(image[1], image[4]), calculate_ratio(image[1], image[7]), calculate_ratio(image[1], image[10])])

    # Min and pool features
    features['band_1_pool_1_min'] = np.min(image[0])
    features['band_2_pool_1_min'] = np.min(image[1])
    features['band_3_pool_1_min'] = np.min(image[2])
    features['band_7_pool_1_min'] = np.min(image[6])

    # LBP feature
    lbp = local_binary_pattern(image [0], 8, 1, method='uniform')
    features['band_1_window_3_lbp_entropy'] = stats.entropy(np.histogram(lbp, bins=256)[0])

    # Wavelet features
    coeffs = pywt.wavedec2(image[0], 'haar', level=3)
    cA3, (cH3, cV3, cD3), (cH2, cV2, cD2), (cH1, cV1, cD1) = coeffs
    features['band_1_wavelet_haar_cD3_min'] = np.min(cD3)

    coeffs = pywt.wavedec2(image[1], 'haar', level=3)
    cA3, (cH3, cV3, cD3), (cH2, cV2, cD2), (cH1, cV1, cD1) = coeffs
    features['band_2_wavelet_haar_cH1_max'] = np.max(cH1)
    features['band_2_wavelet_haar_cD2_max'] = np.max(cD2)
    features['band_2_wavelet_haar_cD3_min'] = np.min(cD3)

    coeffs = pywt.wavedec2(image[2], 'haar', level=3)
    cA3, (cH3, cV3, cD3), (cH2, cV2, cD2), (cH1, cV1, cD1) = coeffs
    features['band_3_wavelet_haar_cH1_max'] = np.max(cH1)

    coeffs = pywt.wavedec2(image[3], 'haar', level=3)
    cA3, (cH3, cV3, cD3), (cH2, cV2, cD2), (cH1, cV1, cD1) = coeffs
    features['band_4_wavelet_haar_cD1_std'] = np.std(cD1)

    coeffs = pywt.wavedec2(image[5], 'haar', level=3)
    cA3, (cH3, cV3, cD3), (cH2, cV2, cD2), (cH1, cV1, cD1) = coeffs
    features['band_6_wavelet_haar_cH2_max'] = np.max(cH2)

    coeffs = pywt.wavedec2(image[6], 'haar', level=3)
    cA3, (cH3, cV3, cD3), (cH2, cV2, cD2), (cH1, cV1, cD1) = coeffs
    features['band_7_wavelet_haar_cV2_max'] = np.max(cV2)

    coeffs = pywt.wavedec2(image[11], 'haar', level=3)
    cA3, (cH3, cV3, cD3), (cH2, cV2, cD2), (cH1, cV1, cD1) = coeffs
    features['band_12_wavelet_haar_cV1_std'] = np.std(cV1)

    coeffs = pywt.wavedec2(image[12], 'haar', level=3)
    cA3, (cH3, cV3, cD3), (cH2, cV2, cD2), (cH1, cV1, cD1) = coeffs
    features['band_13_wavelet_haar_cV1_min'] = np.min(cV1)
    features['band_13_wavelet_haar_cD1_std'] = np.std(cD1)
    features['band_13_wavelet_haar_cV2_std'] = np.std(cV2)
    features['band_13_wavelet_haar_cD2_std'] = np.std(cD2)
    features['band_13_wavelet_haar_cD3_std'] = np.std(cD3)

    feature_values = np.array(list(features.values()))
    feature_values = np.nan_to_num(feature_values, nan=0.0, posinf=0.0, neginf=0.0)
    
    feature_names = list(features.keys())
    return feature_values,feature_names

In [4]:
import os
import csv
import numpy as np
import rasterio


def process_csv(csv_file, base_path, save_path):
    all_features = []
    all_feature_names = []
    all_labels = []

    with open(csv_file, mode='r') as file:
        reader = csv.reader(file)
        for row in reader:
            file_name, _, label = row
            image_path = os.path.join(base_path, file_name)

            feature_values, feature_names = extract_features(image_path)

            all_features.append(feature_values)
            all_feature_names = feature_names  
            all_labels.append(float(label))  

    features_array = np.array(all_features)
    labels_array = np.array(all_labels)

    np.savez(save_path, features=features_array, feature_names=feature_names, labels=labels_array)
    print(f"Features and labels saved to {save_path}")


csv_file = "/kaggle/input/aerosol-optical-depth/train_answer.csv" 
base_path = "/kaggle/input/aerosol-optical-depth/train_images/train_images"  
save_path = "extracted_features_with_labels.npz" 

process_csv(csv_file, base_path, save_path)


/opt/conda/lib/python3.10/site-packages/skimage/feature/texture.py:360: UserWarning: Applying `local_binary_pattern` to floating-point images may give unexpected results when small numerical differences between adjacent pixels are present. It is recommended to use this function with images of integer dtype.
  warnings.warn(
/tmp/ipykernel_22/1590598485.py:16: RuntimeWarning: invalid value encountered in log1p
  features['ratio_band_4_10_min_log'] = np.log1p(np.min(calculate_ratio(image[3], image[9])))


Features and labels saved to extracted_features_with_labels.npz


In [5]:
import os
import numpy as np
import rasterio
import scipy

def process_folder(base_path, save_path):
    all_features = []
    all_feature_names = []
    file_names = []

    for file_name in os.listdir(base_path):
        if file_name.lower().endswith(('.tif', '.tiff')):
            image_path = os.path.join(base_path, file_name)
            feature_values, feature_names = extract_features(image_path)
            all_features.append(feature_values)
            all_feature_names = feature_names  
            file_names.append(file_name)

    features_array = np.array(all_features)

    np.savez(save_path, 
             features=features_array, 
             feature_names=all_feature_names, 
             file_names=file_names)
    
    print(f"Features saved to {save_path}")
    print(f"Number of images processed: {len(file_names)}")
    print(f"Number of features per image: {len(all_feature_names)}")

def load_features(save_path):
    loaded_data = np.load(save_path, allow_pickle=True)
    features = loaded_data['features']
    feature_names = loaded_data['feature_names']
    file_names = loaded_data['file_names']
    return features, feature_names, file_names

base_path = "/kaggle/input/aerosol-optical-depth/test_images/test_images" 
save_path = "extracted_test_features.npz" 

process_folder(base_path, save_path)


/tmp/ipykernel_22/1590598485.py:16: RuntimeWarning: invalid value encountered in log1p
  features['ratio_band_4_10_min_log'] = np.log1p(np.min(calculate_ratio(image[3], image[9])))


Features saved to extracted_test_features.npz
Number of images processed: 1489
Number of features per image: 36


In [6]:
def load_features(save_path):
    loaded_data = np.load(save_path, allow_pickle=True)
    features = loaded_data['features']
    feature_names = loaded_data['feature_names']
    file_names = loaded_data['file_names']
    return features, feature_names, file_names

# Example usage to load the saved test features
features, feature_names, file_names = load_features('extracted_test_features.npz')

print("Feature Names:", feature_names)
print("Shape of Features:", features.shape)
print("File Names:", file_names[:5])  # Display the first 5 file names

Feature Names: ['ratio_band_1_8_mean_max_ratio' 'min_max_ratio_3' 'ratio_band_1_3_2_std'
 'ratio_band_3_5_1_3_diff' 'ratio_band_2_4_1_2_diff'
 'ratio_band_2_3_mean_std_ratio' 'ratio_band_4_10_min_log'
 'ratio_band_1_8_max_mean_diff' 'ratio_band_3_5_1_3_std'
 'band_1_2_3_4_correlation' 'ratio_band_12_13_std' 'band_6_13_std_ratio'
 'ratio_band_1_7_13_max_std_ratio' 'band_3_10_min_mean_diff'
 'ratio_band_1_9_std_min_diff' 'ratio_band_3_5_mean'
 'ratio_band_2_5_8_11_mean' 'band_1_pool_1_min' 'band_2_pool_1_min'
 'band_3_pool_1_min' 'band_7_pool_1_min' 'band_1_window_3_lbp_entropy'
 'band_1_wavelet_haar_cD3_min' 'band_2_wavelet_haar_cH1_max'
 'band_2_wavelet_haar_cD2_max' 'band_2_wavelet_haar_cD3_min'
 'band_3_wavelet_haar_cH1_max' 'band_4_wavelet_haar_cD1_std'
 'band_6_wavelet_haar_cH2_max' 'band_7_wavelet_haar_cV2_max'
 'band_12_wavelet_haar_cV1_std' 'band_13_wavelet_haar_cV1_min'
 'band_13_wavelet_haar_cD1_std' 'band_13_wavelet_haar_cV2_std'
 'band_13_wavelet_haar_cD2_std' 'band_13_wavel

# Catboost Regresssion Head

In [7]:
import numpy as np
import pandas as pd
from catboost import CatBoostRegressor
from sklearn.model_selection import KFold
from scipy.stats import pearsonr
from sklearn.preprocessing import StandardScaler

In [8]:
# Function to load training data
def load_features(save_path):
    loaded_data = np.load(save_path, allow_pickle=True)
    return loaded_data['features'], loaded_data['labels'], loaded_data['feature_names']

# Function to load test data
def load_test_data(test_path):
    loaded_data = np.load(test_path, allow_pickle=True)
    return loaded_data['features'], loaded_data['file_names']


def pearson_corr(y_true, y_pred):
    return pearsonr(y_true, y_pred)[0]
def perform_cross_validation_and_predict(features, labels, test_features, params, n_folds=17):
    kf = KFold(n_splits=n_folds, shuffle=True, random_state=979)
    val_correlations = []
    test_predictions = np.zeros(len(test_features))
    
    scaler = StandardScaler()
    features_scaled = scaler.fit_transform(features)
    test_features_scaled = scaler.transform(test_features)
    
    for fold, (train_index, val_index) in enumerate(kf.split(features_scaled), 1):
        X_train, X_val = features_scaled[train_index], features_scaled[val_index]
        y_train, y_val = labels[train_index], labels[val_index]
        
        model = CatBoostRegressor(**params)
        model.fit(X_train, y_train, eval_set=(X_val, y_val), use_best_model=True, early_stopping_rounds=50, verbose=0)
        
        y_val_pred = model.predict(X_val)
        val_corr = pearson_corr(y_val, y_val_pred)
        val_correlations.append(val_corr)
        
        print(f"Fold {fold} validation correlation: {val_corr:.4f}")
        
        # Make predictions on the test set
        test_predictions += model.predict(test_features_scaled) / n_folds
    
    mean_correlation = np.mean(val_correlations)
    std_correlation = np.std(val_correlations)
    print(f"\nMean validation correlation: {mean_correlation:.4f} (±{std_correlation:.4f})")
    
    return mean_correlation, std_correlation, test_predictions

In [9]:
# Load training features
save_path = "/kaggle/working/extracted_features_with_labels.npz"
features, labels, feature_names = load_features(save_path)

# Load test features
test_path = "/kaggle/working/extracted_test_features.npz"
test_features, file_names = load_test_data(test_path)

# Check the loaded data
print("Training Features Shape:", features.shape)
print("Test Features Shape:", test_features.shape)
print("First 5 Test File Names:", file_names[:5])

Training Features Shape: (4465, 36)
Test Features Shape: (1489, 36)
First 5 Test File Names: ['test_677.tif' 'test_100.tif' 'test_57.tif' 'test_264.tif'
 'test_1243.tif']


In [10]:
# Parameters
params = {
    'learning_rate': 0.0095416806602574589,
    'l2_leaf_reg': 0.04628983043127334, 
    'border_count': 139,
    'bagging_temperature': 3.980762682227694,
    'random_strength': 1.9451355776681154e-08, 
    'min_data_in_leaf': 1,
    'max_leaves': 178,
    'depth': 14,
    'iterations': 4343,
    'task_type': 'GPU',
    'devices': '0',
    'eval_metric': 'RMSE',
    'grow_policy': 'Lossguide',
    'verbose': 0
}

In [11]:
# Perform cross-validation and make predictions
print("Performing cross-validation and making predictions...")

mean_corr, std_corr, test_predictions = perform_cross_validation_and_predict(features, labels, test_features, params)


print("Cross-validation and prediction completed.")

Performing cross-validation and making predictions...
Fold 1 validation correlation: 0.9477
Fold 2 validation correlation: 0.9001
Fold 3 validation correlation: 0.9613
Fold 4 validation correlation: 0.8859
Fold 5 validation correlation: 0.9896
Fold 6 validation correlation: 0.9572
Fold 7 validation correlation: 0.9313
Fold 8 validation correlation: 0.9799
Fold 9 validation correlation: 0.9787
Fold 10 validation correlation: 0.9567
Fold 11 validation correlation: 0.9744
Fold 12 validation correlation: 0.9508
Fold 13 validation correlation: 0.9683
Fold 14 validation correlation: 0.9778
Fold 15 validation correlation: 0.9618
Fold 16 validation correlation: 0.9500
Fold 17 validation correlation: 0.9809

Mean validation correlation: 0.9560 (±0.0273)
Cross-validation and prediction completed.


In [12]:
# Save predictions to CSV
output_df = pd.DataFrame({'file_name': file_names, 'prediction': test_predictions})
output_df.to_csv('C_PD_9F.csv', index=False, header=False)
print("Predictions saved to 'C_PD_9F.csv'")

Predictions saved to 'C_PD_9F.csv'


In [13]:
output_df.head(10)

,file_name,prediction
0,test_677.tif,0.053104
1,test_100.tif,0.128500
2,test_57.tif,0.097586
3,test_264.tif,0.051345
4,test_1243.tif,0.040634
5,test_306.tif,0.120217
6,test_1360.tif,1.095037
7,test_1476.tif,0.256265
8,test_336.tif,0.131005
9,test_711.tif,0.340944
